In [32]:
'''
Author: Suryakant Upadhayay
Purpose: This script is a sample demonstration to create and validate cloud optimized GeoTiff (COG)  file using:
    1. From RGB GeoTiff (created here using any 3 bands of the provided remote sensing data)
    2. From multi/hyper spectral bands remote sensing data
    
'''

from osgeo import gdal, osr
import os
import io
import numpy as np
import glob

def createRgbTiff(newRasterFilePath,array,format='GTiff'):

    bands = array.shape[0]
    rows = array.shape[1]
    cols = array.shape[2]
    rasterOrigin=[0,0]
    pixelWidth = 1
    pixelHeight = 1
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]
    driver = gdal.GetDriverByName(format)
    #Here is where I assign three bands to the raster with int 3
    options = ['PHOTOMETRIC=RGB', 'PROFILE=GeoTIFF']
    outRaster = driver.Create(newRasterFilePath, cols, rows, bands, gdal.GDT_UInt16, options=options)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    #outband = outRaster.GetRasterBand(1)
    #outband.WriteArray(array)
    for band in range(bands):
        outRaster.GetRasterBand(band+1).WriteArray( array[band, :, :] )

    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())

    outRaster.GetRasterBand(1).SetColorInterpretation(gdal.GCI_RedBand)
    outRaster.GetRasterBand(2).SetColorInterpretation(gdal.GCI_GreenBand)
    outRaster.GetRasterBand(3).SetColorInterpretation(gdal.GCI_BlueBand)
    outRaster.FlushCache()
    print('RGB COG creation successful')

def createCOG(newRasterfn,array):

    cols = array.shape[2]
    rows = array.shape[1]
    nbands = array.shape[0]
    driver = gdal.GetDriverByName('MEM')
    data_set = driver.Create('', cols, rows, nbands,
                             gdal.GDT_Float32)

    for i in range(nbands):
        data_set.GetRasterBand(i + 1).WriteArray(array[i])

    data = None
    data_set.BuildOverviews("NEAREST", [2, 4, 8, 16, 32, 64])

    driver = gdal.GetDriverByName('GTiff')
    data_set2 = driver.CreateCopy(newRasterfn, data_set,
                                  options=["COPY_SRC_OVERVIEWS=YES",
                                           "TILED=YES",
                                           "COMPRESS=LZW"])
    data_set = None
    data_set2 = None


def readSampleRgbBands():
    print("Reading raster bands for RGB")
    #reading and combining any 3 tiffs to create a raster RGB band array, in this example reading band index 7, 8, 9
    rgb_array = np.stack([gdal.Open(tiffs[7]).ReadAsArray(),
                          gdal.Open(tiffs[8]).ReadAsArray(),
                          gdal.Open(tiffs[9]).ReadAsArray()])
    print("Reading raster bands successful")
    return rgb_array

if __name__ == "__main__":
     
    #path to the folder which contains different bands data i.e tif files in this case    
    path = r'C:\Users\Suryakant Upadhayay\Downloads\EO1H1430452010208110Kt_1GST\EO1H1430452010208110Kt'
    tiffs = glob.glob(os.path.join(path,"*.tif*"))  #getting list of all the available tiffs in the folder

    
    ##creating and validating COG using RGB first 
    
    sampleRGBArray = readSampleRgbBands()
    #output RGB COG file path
    sampleOutputPath = r'C:\Users\Suryakant Upadhayay\Downloads\Sample_RGB.tif'
    createRgbTiff(sampleOutputPath,sampleRGBArray)
    print("Creating COG from sample RGB")
    newRasterfn = r'C:\Users\Suryakant Upadhayay\Downloads\Sample_RGB_COG.tif'
    array = gdal.Open(sampleOutputPath).ReadAsArray()
    print(array.shape)
    createCOG(newRasterfn,array)
    print("Validating created Sample RGB COG")
    #there is script Test_COG.py to validate COG, calling the script with script and COG paths as arguments
    %run -i "C:\Users\Suryakant Upadhayay\Downloads\Test_COG.py" "C:\Users\Suryakant Upadhayay\Downloads\Sample_RGB_COG.tif"
    
    ##creating and validating COG using multi/hyper spectral bands remote sensing data
    print("Reading multi/hyper spectral bands")
    arrs = []
    for tiff in tiffs:
        B = gdal.Open(tiff)
        b = B.ReadAsArray()
        arrs.append(b)
    
    array = np.stack(arrs)
    print(array.shape, array.ndim)
    print("Creating COG from multi/hyper spectral bands")
    newRasterfn = r'C:\Users\Suryakant Upadhayay\Downloads\Sample_COG.tif'
    createCOG(newRasterfn,array)
    print("Validating created Sample COG")
    #there is script Test_COG.py to validate COG, calling the script with script and COG paths as arguments
    %run -i "C:\Users\Suryakant Upadhayay\Downloads\Test_COG.py" "C:\Users\Suryakant Upadhayay\Downloads\Sample_COG.tif"
    
    
    print("Finished")



Reading raster bands for RGB
Reading raster bands successful
RGB COG creation successful
Creating COG from sample RGB
(3, 3521, 1061)
Validating created Sample RGB COG
C:\Users\Suryakant Upadhayay\Downloads\Sample_RGB_COG.tif is a valid cloud optimized GeoTIFF

The size of all IFD headers is 2750 bytes
Reading multi/hyper spectral bands
(242, 3521, 1061) 3
Creating COG from multi/hyper spectral bands
Validating created Sample COG
C:\Users\Suryakant Upadhayay\Downloads\Sample_COG.tif is a valid cloud optimized GeoTIFF

The size of all IFD headers is 12816 bytes
Finished
